In [2]:
import copy
import glob
import random
import regex as re
import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
%load_ext rpy2.ipython
%run ../airquality/data/gen_daily_targets.py
%run ../airquality/data/prepare_data.py

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
test = pd.read_csv('../data/targets.csv')
stations = pd.read_csv('../data/stations.csv')
extra_features = pd.read_csv('../data/dataset_v1.csv', index_col=0).drop([
    'holiday', 'target', 
], axis=1)
rolling_mean_features = pd.read_csv('../data/data_roll_day_conc_obs.csv')
extra_features = extra_features[[c for c in extra_features.columns if not 'dist' in c]]

In [5]:
observations = pd.read_csv('../data/observations.csv', index_col=0)
models = pd.read_csv('../data/models.csv', index_col=0)
obs_and_mods = pd.merge(models, observations, how='left',
                    on=['station', 'day', 'time', 'datetime' ,'year'],
                    suffixes=('_mod', '')).sort_values('datetime', ascending=True)
obs_and_mods.to_csv('../data/obs_and_mod.csv')
obs_and_mods_cols = ['pred_0_days', 'pred_1_days', 'Concentration', 'target', 'day']
obs_and_mods.head()

,pred_0_days,pred_1_days,day,lon,lat,year,station,datetime,time,Concentration,target
0,38.79,34.11,2013-01-01,2.15,41.39,2013,ES1438A,2013-01-01 00:00:00,00:00:00,nan,nan
1,28.53,27.48,2013-01-01,2.13,41.38,2013,ES1396A,2013-01-01 00:00:00,00:00:00,nan,nan
2,35.85,42.57,2013-01-01,2.20,41.40,2013,ES0691A,2013-01-01 00:00:00,00:00:00,nan,nan
3,31.81,31.59,2013-01-01,2.15,41.40,2013,ES1480A,2013-01-01 00:00:00,00:00:00,nan,nan
4,31.81,31.59,2013-01-01,2.15,41.43,2013,ES1856A,2013-01-01 00:00:00,00:00:00,nan,nan


In [6]:
agg_types = ['mean', 'max', 'std']
tall_series = obs_and_mods.groupby(['day', 'station']).agg({
    'Concentration': agg_types
})['Concentration'].reset_index()
aggs = [tall_series.pivot(index='day', columns='station', values=agg) for agg in agg_types]
aggs = [df.rename(columns={c: c + '_' + agg for c in df.columns}) for df, agg in zip(aggs, agg_types)]
wide_series = pd.concat(aggs, axis=1)
wide_series.head()

station,ES0691A_mean,ES1396A_mean,ES1438A_mean,ES1480A_mean,ES1679A_mean,ES1856A_mean,ES1992A_mean,ES0691A_max,ES1396A_max,ES1438A_max,...,ES1679A_max,ES1856A_max,ES1992A_max,ES0691A_std,ES1396A_std,ES1438A_std,ES1480A_std,ES1679A_std,ES1856A_std,ES1992A_std
day,,,,,,,,,,,,,,,,,,,,,
2013-01-01,42.39,38.61,51.39,43.87,37.13,14.78,23.35,72.00,98.00,112.00,...,69.00,39.00,68.00,18.13,26.67,25.73,24.63,20.29,10.99,21.15
2013-01-02,52.50,47.58,63.71,54.50,48.75,32.08,43.29,87.00,86.00,119.00,...,76.00,77.00,85.00,16.61,24.72,26.48,19.18,16.91,25.43,26.53
2013-01-03,64.62,54.62,74.04,64.17,49.25,38.88,49.04,119.00,85.00,117.00,...,78.00,94.00,110.00,22.69,19.05,22.87,21.70,14.87,22.17,31.43
2013-01-04,54.12,44.30,44.50,89.13,61.92,22.18,39.04,80.00,113.00,123.00,...,108.00,60.00,99.00,16.58,28.35,32.46,45.41,25.83,16.98,28.20
2013-01-05,36.88,39.17,nan,89.00,64.00,12.25,33.54,60.00,114.00,nan,...,114.00,16.00,85.00,11.00,22.21,nan,21.63,17.04,2.56,17.75


## Cross-Validation SVM

In [40]:
features['Concentration'].head()

day
2013-01-01    98.00
2013-01-02    86.00
2013-01-03    85.00
2013-01-04   113.00
2013-01-05   114.00
Name: Concentration, dtype: float64

In [62]:
X.shape

(730, 2865)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import log_loss, make_scorer

lags = 100
for s in test.station.unique():
    data = wide_series[list(wide_series.reset_index().day < '2015-01-01')]
    to_lag = data[[c for c in data.columns if not c in ['{}_{}'.format(s, agg) for agg in agg_types]]]
    features = create_lagged_features(to_lag, lags)\
        .join(extra_features[extra_features.station == s].set_index('date'))\
        .join(rolling_mean_features[rolling_mean_features.station == s]
              .set_index('date').drop(['station', 'max_conc_obs'], axis=1))\
        .join(obs_and_mods[obs_and_mods.station == s][['Concentration', 'day']].groupby('day').max())\
        .fillna(method='ffill').fillna(0)
    X = features[[c for c in features.columns if not c in [
        'time', 'datetime', 'Concentration', 'target', 'station'
    ]]].values[:, np.where(X[0] != 98)[0]]
    y = (features['Concentration'].fillna(method='ffill').values > 100).astype(int)

    params = pd.DataFrame({
        'C': np.random.choice(np.linspace(0.01, .99, 100), 15),
        'score': [np.nan] * 15
    })
    for i, r in params[['C']].iterrows():
        kf = KFold(n_splits=3)
        metric = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            p = dict(r)
            p['probability'] = True
            try:
                model = SVC(**p)
                model.fit(X_train, y_train)
                y_pred = model.predict_proba(X_test)
                metric.append(
                    log_loss(y_test, y_pred, labels=(0, 1))
                ) 
            except ValueError as e: 
                print(e)
                metric.append(np.nan)
                pass

        params.loc[i, 'score'] = np.mean(metric)
        print(params.iloc[i])

    params.to_csv('../reports/cv_ssvm_Jonas_8_{}_{}.csv'
                  .format(s, len(glob.glob('../reports/cv_ssvm_Jonas_8_{}_*'.format(s)))))
    params.head()

C       0.09
score   0.28
Name: 0, dtype: float64
C       0.87
score   0.28
Name: 1, dtype: float64
C       0.76
score   0.28
Name: 2, dtype: float64
C       0.37
score   0.28
Name: 3, dtype: float64
C       0.46
score   0.28
Name: 4, dtype: float64
C       0.89
score   0.28
Name: 5, dtype: float64
C       0.56
score   0.28
Name: 6, dtype: float64
C       0.81
score   0.28
Name: 7, dtype: float64
C       0.02
score   0.28
Name: 8, dtype: float64
C       0.03
score   0.28
Name: 9, dtype: float64
C       0.45
score   0.28
Name: 10, dtype: float64
C       0.49
score   0.28
Name: 11, dtype: float64
C       0.37
score   0.28
Name: 12, dtype: float64
C       0.56
score   0.28
Name: 13, dtype: float64
C       0.86
score   0.28
Name: 14, dtype: float64
C       0.68
score   0.31
Name: 0, dtype: float64
C       0.76
score   0.31
Name: 1, dtype: float64
C       0.11
score   0.31
Name: 2, dtype: float64
C       0.18
score   0.31
Name: 3, dtype: float64
C       0.50
score   0.31
Name: 4, dtype: flo

## SVM Classification per Station

* Targeting `target`
* Using imputed for 2015

In [20]:
tall_series_mod = obs_and_mods.fillna(0).groupby(['day', 'station']).agg({
    'pred_0_days': agg_types
})['pred_0_days'].reset_index().rename(columns={
    'pred_0_days': 'Concentration'
})
aggs = [tall_series_mod.pivot(index='day', columns='station', values=agg) for agg in agg_types]
aggs = [df.rename(columns={c: c + '_' + agg for c in df.columns}) for df, agg in zip(aggs, agg_types)]
wide_series_mod = pd.concat(aggs, axis=1)
wide_series_mod.tail()

station,ES0691A_mean,ES1396A_mean,ES1438A_mean,ES1480A_mean,ES1679A_mean,ES1856A_mean,ES1992A_mean,ES0691A_max,ES1396A_max,ES1438A_max,...,ES1679A_max,ES1856A_max,ES1992A_max,ES0691A_std,ES1396A_std,ES1438A_std,ES1480A_std,ES1679A_std,ES1856A_std,ES1992A_std
day,,,,,,,,,,,,,,,,,,,,,
2015-12-27,51.54,50.02,57.79,56.64,57.79,56.64,50.02,104.21,96.41,104.76,...,104.76,108.53,96.41,25.52,28.88,23.64,31.42,23.64,31.42,28.88
2015-12-28,69.50,59.83,74.54,70.96,74.54,70.96,59.83,123.89,116.91,135.50,...,135.50,131.59,116.91,28.95,26.06,31.27,28.83,31.27,28.83,26.06
2015-12-29,48.35,44.61,53.75,46.21,53.75,46.21,44.61,93.35,113.39,106.20,...,106.20,113.48,113.39,23.33,28.05,22.57,28.15,22.57,28.15,28.05
2015-12-30,55.49,43.79,54.16,49.88,54.16,49.88,43.79,103.20,91.71,95.34,...,95.34,105.85,91.71,27.95,24.97,23.91,29.40,23.91,29.40,24.97
2015-12-31,64.10,51.87,64.55,60.12,64.55,60.12,51.87,103.42,97.83,109.93,...,109.93,107.90,97.83,22.91,22.19,23.95,25.89,23.95,25.89,22.19


In [21]:
to_impute = wide_series.loc[test['date'].unique()]
for s in obs_and_mods.station.unique():
    columns = [s + '_' + agg for agg in agg_types]
    to_impute[columns] = wide_series_mod.loc[to_impute.reset_index().day, columns]
    
originals = wide_series[list(~wide_series.reset_index().day.isin(test['date'].unique()))]
wide_series_imputed = pd.concat([to_impute, originals]).sort_index()
wide_series_imputed.head()

station,ES0691A_mean,ES1396A_mean,ES1438A_mean,ES1480A_mean,ES1679A_mean,ES1856A_mean,ES1992A_mean,ES0691A_max,ES1396A_max,ES1438A_max,...,ES1679A_max,ES1856A_max,ES1992A_max,ES0691A_std,ES1396A_std,ES1438A_std,ES1480A_std,ES1679A_std,ES1856A_std,ES1992A_std
day,,,,,,,,,,,,,,,,,,,,,
2013-01-01,42.39,38.61,51.39,43.87,37.13,14.78,23.35,72.00,98.00,112.00,...,69.00,39.00,68.00,18.13,26.67,25.73,24.63,20.29,10.99,21.15
2013-01-02,52.50,47.58,63.71,54.50,48.75,32.08,43.29,87.00,86.00,119.00,...,76.00,77.00,85.00,16.61,24.72,26.48,19.18,16.91,25.43,26.53
2013-01-03,64.62,54.62,74.04,64.17,49.25,38.88,49.04,119.00,85.00,117.00,...,78.00,94.00,110.00,22.69,19.05,22.87,21.70,14.87,22.17,31.43
2013-01-04,54.12,44.30,44.50,89.13,61.92,22.18,39.04,80.00,113.00,123.00,...,108.00,60.00,99.00,16.58,28.35,32.46,45.41,25.83,16.98,28.20
2013-01-05,36.88,39.17,nan,89.00,64.00,12.25,33.54,60.00,114.00,nan,...,114.00,16.00,85.00,11.00,22.21,nan,21.63,17.04,2.56,17.75


In [22]:
all_frames = []
for s in test.station.unique():
    frames = [pd.read_csv(f, index_col=0) for f in glob.glob('../reports/cv_svm_Jonas_8*{}*'.format(s))]
    try:
        frame = pd.concat(frames)
        frame['station'] = s
        all_frames.append(frame)
    except ValueError:
        pass
    
cv_results = pd.concat(all_frames)
cv_results = cv_results[cv_results['score'] > 0]
cv_results.sort_values(['score', 'station']).drop_duplicates('station')

,nu,score,station
1,0.05,0.27,ES1679A
0,0.13,0.28,ES0691A
6,0.23,0.31,ES1856A
41,0.01,0.31,ES1396A
3,0.05,0.52,ES1992A
2,0.70,0.60,ES1480A
0,0.75,0.63,ES1438A


In [23]:
test_params = []
for i, r in cv_results.sort_values(['score', 'station']).drop_duplicates('station').iterrows():
    p = dict(r)
    del p['score']
    del p['station']
    test_params.append((r['station'], p))
    
test_params

[('ES1679A', {'nu': 0.0495959595959596}),
 ('ES0691A', {'nu': 0.12878787878787878}),
 ('ES1856A', {'nu': 0.22777777777777775}),
 ('ES1396A', {'nu': 0.01}),
 ('ES1992A', {'nu': 0.0495959595959596}),
 ('ES1480A', {'nu': 0.702929292929293}),
 ('ES1438A', {'nu': 0.7524242424242424})]

In [31]:
from sklearn.svm import SVC
from sklearn.model_selection import  KFold
from sklearn.metrics import log_loss

lags = 100
preds_by_station = []
for s, p in test_params:
    data = pd.concat([
        wide_series[list(wide_series.reset_index().day < '2015-01-01')],
        wide_series_imputed[list(wide_series_imputed.reset_index().day >= '2015-01-01')]
    ])
    to_lag = data[[c for c in data.columns if not c in ['{}_{}'.format(s, agg) for agg in agg_types]]]
    features = create_lagged_features(to_lag, lags)\
        .join(extra_features[extra_features.station == s].set_index('date'))\
        .join(rolling_mean_features[rolling_mean_features.station == s]
              .set_index('date').drop(['station', 'max_conc_obs'], axis=1))\
        .join(obs_and_mods[obs_and_mods.station == s][['Concentration', 'day']].groupby('day').max())\
        .fillna(method='ffill')
    X = features[[c for c in features.columns if not c in [
        'time', 'datetime', 'Concentration', 'target', 'station'
    ]]].values
    y = (features['Concentration'].values > 100).astype(int)
    y_pred_all = copy.deepcopy(y).astype(float)
    metrics_all = []
    
    kf = KFold(n_splits=2)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        p['probability'] = True
        model = SVC(**p)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        y_pred_all[test_index] = y_pred
        metrics_all.append(log_loss(y_test, y_pred, labels=(0, 1)))
        
    preds_by_station.append((s, y_pred_all))

predictions = pd.DataFrame(np.column_stack([e[1] for e in preds_by_station]), columns=test.station.unique())
predictions['date'] = features.reset_index()['day']
predictions.to_csv('../reports/pred_svm_Jonas_8_{}.csv'.format(len(glob.glob('../reports/pred_svm_Jonas_8_*'))))
print('CV Metric: {}'.format(np.mean(metrics_all)))
predictions.tail()

ValueError: specified nu is infeasible